# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
csv_folder = '/event_data'

# the the `csv_folder` folder relative to current working directory
filepath = os.getcwd() + csv_folder

# iterate over files in the folder 
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# file in which the .csv files will be merged into
output_csv = 'event_datafile_new.csv'

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skip the header
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# open the `output_csv` file and write the rows
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open(output_csv, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        # skip empty rows
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Import Python packages 

In [5]:
from cassandra.cluster import Cluster
from cql_queries import *

#### Creating a Cluster

In [6]:
# connect to the cluster
cluster = Cluster(['127.0.0.1'])

# open a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# create the keyspace if it still doesn't exist
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# connect to the keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
print(session_table_create)
try:
    session.execute(session_table_create)
except Exception as e:
    print(e)


CREATE TABLE IF NOT EXISTS session_table (
    artist_name text,
    song_name text,
    song_length decimal,
    session_id int,
    item_in_session int,
    user_id int,
    user_first_name text,
    user_last_name text,
    user_gender text,
    user_level text,
    user_location text,
    PRIMARY KEY (session_id,item_in_session)
);



In [10]:
# file to be inserted
file = 'event_datafile_new.csv'

# open csv file
df = pd.read_csv(file)

# for each row on the file
for i, row in df.iterrows():
    # insert into session history table
    session.execute(session_table_insert, tuple(row[[
        'artist',
        'song',
        'length',
        'sessionId',
        'itemInSession',
        'userId',
        'firstName',
        'lastName',
        'gender',
        'level',
        'location'
    ]]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
try:
    rows = session.execute(session_table_select,(338,4))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
print(user_table_create)
try:
    session.execute(user_table_create)
except Exception as e:
    print(e)


CREATE TABLE IF NOT EXISTS user_table (
    artist_name text,
    song_name text,
    song_length decimal,
    session_id int,
    item_in_session int,
    user_id int,
    user_first_name text,
    user_last_name text,
    user_gender text,
    user_level text,
    user_location text,
    PRIMARY KEY (user_id,session_id,item_in_session)
);



In [13]:
# file to be inserted
file = 'event_datafile_new.csv'

# open csv file
df = pd.read_csv(file)

# for each row on the file
for i, row in df.iterrows():
    # insert into user history table
    session.execute(user_table_insert, tuple(row[[
        'artist',
        'song',
        'length',
        'sessionId',
        'itemInSession',
        'userId',
        'firstName',
        'lastName',
        'gender',
        'level',
        'location'
    ]]))

In [14]:
try:
    rows = session.execute(user_table_select,(10,182))
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_name, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
print(songplay_table_create)
try:
    session.execute(songplay_table_create)
except Exception as e:
    print(e)


CREATE TABLE IF NOT EXISTS songplay_table (
    artist_name text,
    song_name text,
    song_length decimal,
    session_id int,
    item_in_session int,
    user_id int,
    user_first_name text,
    user_last_name text,
    user_gender text,
    user_level text,
    user_location text,
    PRIMARY KEY (song_name,artist_name,user_id)
);



In [17]:
# file to be inserted
file = 'event_datafile_new.csv'

# open csv file
df = pd.read_csv(file)

# for each row on the file
for i, row in df.iterrows():
    # insert into songplay history table
    session.execute(songplay_table_insert, tuple(row[[
        'artist',
        'song',
        'length',
        'sessionId',
        'itemInSession',
        'userId',
        'firstName',
        'lastName',
        'gender',
        'level',
        'location'
    ]]))

In [18]:
try:
    rows = session.execute(songplay_table_select%"'All Hands Against His Own'")
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
for query in drop_table_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()